In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
from spliceai.utils import one_hot_encode
import numpy as np
import pandas as pd
import argparse

from Bio import SeqIO
from Bio import motifs

import warnings
warnings.filterwarnings("ignore")

2025-07-08 12:51:20.721772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/storage/westbrook/tmp/ipykernel_3816/640282757.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


In [4]:
exec(open('/storage/thinc/git_repos/deep-learning/utils/utils.py').read())

In [5]:
genome_file = '/storage/thinc/data/genomes/Homo_sapiens/ENSEMBL/92/hg38/WholeGenomeFasta/genome.fa'
genome = read_genome(genome_file)

In [6]:
context = 10000
paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
models = [load_model(resource_filename('spliceai', x)) for x in paths]

In [7]:
def read_junction_table(file):
    coord = pd.read_csv(file, delimiter = "\n", header = None)
    tmp = list(coord[0])
    chr = []
    pos = []
    for x in tmp:
        chr.append(x.split("_")[0])
        pos.append(int(x.split("_")[1]))
    
    coord = pd.DataFrame({"chr": chr, "pos": pos})
    
    return(coord)


def predict_junction(chr, pos):
    input_sequence = genome[chr][(pos - 5000):(pos + 5000)]
    input_sequence = str(input_sequence)

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)

    acceptor_prob = y[0, :, 1]
    donor_prob = y[0, :, 2]
    
    return(acceptor_prob, donor_prob)



In [16]:
coord = read_junction_table("/storage/thinc/home/233454/projects/junctions_from_brian/unannotated_5_dTAG.txt")
coord.head()

,chr,pos
0,10,100002941
1,10,100005071
2,10,100006236
3,10,100154691
4,10,100156064


In [59]:
files = ["unannotated_5_dTAG.txt", "unannotated_3_dTAG.txt", "unannotated_5_Unt.txt", "unannotated_3_Unt.txt"]
path = "/storage/thinc/home/233454/projects/junctions_from_brian/"
files

['unannotated_5_dTAG.txt',
 'unannotated_3_dTAG.txt',
 'unannotated_5_Unt.txt',
 'unannotated_3_Unt.txt']

In [23]:
coord = pd.read_csv("../../data/dtag_experiments/random_500_ujs.csv")
coord = coord[coord["strand"] == "+"]
coord

,chr,start,end,strand,class
3,2,200398997,200412310,+,matching 3
4,5,150535026,150535699,+,matching 3
5,1,161170462,161170619,+,matching 3
6,16,30733450,30733717,+,matching 5
7,1,234374390,234377458,+,matching 5
...,...,...,...,...,...
489,2,36506091,36509982,+,matching 3
492,19,8400059,8401867,+,matching 5
494,2,168456619,168493221,+,matching 5
495,5,33460074,33460901,+,matching 3


In [25]:
acceptor_probs_all = []
donor_probs_all = []

for x in range(coord.shape[0]):
        #print(x)

    chr = coord.iloc[x, :][0]
    pos = int(coord.iloc[x, :][1])

    acceptor_prob, donor_prob = predict_junction(chr, pos)

    acceptor_probs_all.append(acceptor_prob)
    donor_probs_all.append(donor_prob)
    
acceptor_probs_all = np.vstack(acceptor_probs_all)
donor_probs_all = np.vstack(donor_probs_all)

acc_start = acceptor_probs_all[:,4900:5100]
don_start = donor_probs_all[:,4900:5100]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 884ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 924ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [26]:
np.savetxt(fname = "../../data/dtag_experiments/acceptor_spliceai_start.txt",
           X = acc_start)
    
np.savetxt(fname = "../../data/dtag_experiments/donor_spliceai_start.txt",
           X = don_start)

In [27]:
acceptor_probs_all = []
donor_probs_all = []

for x in range(coord.shape[0]):
        #print(x)

    chr = coord.iloc[x, :][0]
    pos = int(coord.iloc[x, :][2])

    acceptor_prob, donor_prob = predict_junction(chr, pos)

    acceptor_probs_all.append(acceptor_prob)
    donor_probs_all.append(donor_prob)
    
acceptor_probs_all = np.vstack(acceptor_probs_all)
donor_probs_all = np.vstack(donor_probs_all)

acc_end = acceptor_probs_all[:,4900:5100]
don_end = donor_probs_all[:,4900:5100]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [28]:
np.savetxt(fname = "../../data/dtag_experiments/acceptor_spliceai_end.txt",
           X = acc_end)
    
np.savetxt(fname = "../../data/dtag_experiments/donor_spliceai_end.txt",
           X = don_end)